In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import dask.dataframe as dd

In [2]:
# train_data_section = pd.read_csv("data/train.csv", iterator=False, chunksize=1024)
# train = train_data_section.get_chunk()
# train.head()

In [3]:
df_train = pd.read_csv(
    'data/train.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

In [4]:
items = pd.read_csv(
    "data/items.csv",
).set_index("item_nbr")

In [5]:
df_2017 = df_train[df_train.date.isin(
    pd.date_range("2017-05-31", periods=7 * 11))].copy()
del df_train

In [14]:
promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)

In [15]:
promo_2017_train.head()

onpromotion                                              \
date                2017-05-31 2017-06-01 2017-06-02 2017-06-03 2017-06-04   
store_nbr item_nbr                                                           
1         96995          False      False      False      False      False   
          99197          False      False      False      False      False   
          103520         False      False      False      False      False   
          103665         False      False      False      False      False   
          105574         False      False      False      False      False   

                                                                           \
date               2017-06-05 2017-06-06 2017-06-07 2017-06-08 2017-06-09   
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   
          105574        False      False      False      False      False   

                      ...                                                  \
date                  ...     2017-08-06 2017-08-07 2017-08-08 2017-08-09   
store_nbr item_nbr    ...                                                   
1         96995       ...          False      False      False      False   
          99197       ...          False      False      False      False   
          103520      ...          False      False      False      False   
          103665      ...          False      False      False      False   
          105574      ...          False      False      False      False   

                                                                           \
date               2017-08-10 2017-08-11 2017-08-12 2017-08-13 2017-08-14   
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   
          105574        False      False      False      False      False   

                               
date               2017-08-15  
store_nbr item_nbr             
1         96995         False  
          99197         False  
          103520        False  
          103665        False  
          105574        False  

[5 rows x 77 columns]

In [18]:
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)

In [19]:
promo_2017_train

date                2017-05-31  2017-06-01  2017-06-02  2017-06-03  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   
          105575         False       False       False       False   
          105577         False       False       False       False   
          105693         False       False       False       False   
          105737         False       False       False       False   
          105857         False       False       False       False   
          106716         False       False       False       False   
          108079         False       False       False       False   
          108634         False       False       False       False   
          108696         False       False        True       False   
          108698         False       False        True       False   
          108701          True       False        True       False   
          108786         False       False       False       False   
          108797         False       False       False       False   
          108862         False       False       False       False   
          108952         False       False       False       False   
          111223         False       False       False       False   
          111397          True       False       False       False   
          112830         False       False       False       False   
          114778         False       False       False       False   
          114790         False       False       False        True   
          114799         False       False       False       False   
          114800         False       False       False       False   
          115267         False       False       False       False   
          115611         False       False       False       False   
          115693         False       False        True        True   
...                        ...         ...         ...         ...   
54        2081056        False       False       False       False   
          2081064        False       False       False       False   
          2081095        False       False       False       False   
          2081142        False       False       False       False   
          2081161        False       False       False       False   
          2083837        False       False       False       False   
          2083859        False       False       False       False   
          2084278        False       False       False       False   
          2084557        False       False        True        True   
          2086882         True       False       False       False   
          2087374        False       False       False       False   
          2087409        False       False       False       False   
          2087543        False       False       False       False   
          2087567        False       False       False       False   
          2087933        False       False       False       False   
          2087978        False       False       False        True   
          2088922        False       False       False       False   
          2089036        False       False       False       False   
          2089339        False       False       False       False   
          2099230        False       False       False        True   
          2101795        False       False       False       False   
          2103250        False       False       False       False   
          2105347         True        True        True        True   
          2106464        False       False       False       False   
          

In [21]:
df_test = pd.read_csv(
    "data/test.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

In [22]:
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)

In [23]:
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)

In [25]:
del promo_2017_test, promo_2017_train

In [26]:
df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)

In [28]:
items = pd.read_csv(
    "data/items.csv",
).set_index("item_nbr")

In [30]:
df_2017.index.get_level_values(1)

Int64Index([  96995,   99197,  103520,  103665,  105574,  105575,  105577,
             105693,  105737,  105857,
            ...
            2101795, 2103250, 2105347, 2106464, 2108569, 2109909, 2110456,
            2113343, 2113914, 2116416],
           dtype='int64', name='item_nbr', length=156790)

In [32]:
items = items.reindex(df_2017.index.get_level_values(1))

In [35]:
def get_timespan(df, dt, minus, periods):
    return df[
        pd.date_range(dt - timedelta(days=minus), periods=periods)
    ]

In [42]:
from datetime import date, timedelta
def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "mean_3_2017": get_timespan(df_2017, t2017, 3, 3).mean(axis=1).values,
        "mean_7_2017": get_timespan(df_2017, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(df_2017, t2017, 14, 14).mean(axis=1).values,
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values
    })
    for i in range(16):
        X["promo_{}".format(i)] = promo_2017[
            t2017 + timedelta(days=i)].values.astype(np.uint8)
    if is_train:
        y = df_2017[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X

In [43]:
t2017 = date(2017, 6, 21)
X_l, y_l = [], []
for i in range(4):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(
        t2017 + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)

In [46]:
X_train.shape

(627160, 20)

In [47]:
y_train.shape

(627160, 16)

In [48]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import dask.dataframe as dd
from toolbox.data_aggregation import *
from toolbox.linear_reg import *
import datetime
import numpy as np

In [50]:
test_dd = dd.read_csv("data/test.csv", parse_dates=["date"])

In [55]:
time_range_min = test_dd['date'].min().compute()
time_range_max = test_dd['date'].max().compute()

In [56]:
print(time_range_min, time_range_max)

2017-08-16 2017-08-31
